In [2]:
import requests

from bs4 import BeautifulSoup

In [3]:
response = requests.get(
    'https://www.sofascore.com/football/match/adana-demirspor-fenerbahce/clbsbmb#id:12528186',
    headers={'User-Agent': 'Mozilla/5.0'} # you'll be blocked if you don't use some type of user agent
)

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')


In [5]:
soup.select('g[cursor="pointer"]')


[]

In [17]:
headers = {
    'authority': 'api.sofascore.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'dnt': '1',
    'if-none-match': 'W/"4bebed6144"',
    'origin': 'https://www.sofascore.com',
    'referer': 'https://www.sofascore.com/',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

# you'll need to change the url to match the one you copied. As you can see they have some sort of match id to identify the game --> 10385636
response = requests.get('https://api.sofascore.com/api/v1/event/12117217/shotmap', headers=headers)

In [18]:
response

<Response [200]>

In [19]:
shots = response.json()


In [20]:
shots

{'shotmap': [{'player': {'name': 'André Silva',
    'slug': 'andre-silva',
    'shortName': 'A. Silva',
    'position': 'F',
    'jerseyNumber': '17',
    'userCount': 1263,
    'id': 925072},
   'isHome': True,
   'shotType': 'miss',
   'situation': 'assisted',
   'playerCoordinates': {'x': 0.7885750321085476,
    'y': 20.704843570335925,
    'z': 0},
   'bodyPart': 'right-foot',
   'goalMouthLocation': 'close-left',
   'goalMouthCoordinates': {'x': 0, 'y': 20.26772880765766, 'z': 26.4},
   'xg': 0.124048627913,
   'xgot': 0,
   'id': 3706976,
   'time': 95,
   'addedTime': 5,
   'timeSeconds': 5671,
   'draw': {'start': {'x': 45.287443154254404, 'y': 0.1911944812343378},
    'end': {'x': 15.85546507314605, 'y': 0},
    'goal': {'x': 86.38495308244313, 'y': 17.034199991098703}},
   'reversedPeriodTime': 1,
   'reversedPeriodTimeSeconds': 629,
   'incidentType': 'shot'},
  {'player': {'name': 'Welington',
    'firstName': '',
    'lastName': '',
    'slug': 'welington',
    'shortName'

In [21]:
type(shots)

dict

In [22]:
shots.keys()

dict_keys(['shotmap'])

In [23]:
shotmap_value = shots['shotmap']
print(type(shotmap_value))

<class 'list'>


In [24]:
if isinstance(shotmap_value, list):
    print(shotmap_value[:5])  # İlk 5 elemanı gösterir.
elif isinstance(shotmap_value, dict):
    print(list(shotmap_value.items())[:5])  # İlk 5 anahtar-değer çiftini gösterir.
else:
    print(shotmap_value) 

[{'player': {'name': 'André Silva', 'slug': 'andre-silva', 'shortName': 'A. Silva', 'position': 'F', 'jerseyNumber': '17', 'userCount': 1263, 'id': 925072}, 'isHome': True, 'shotType': 'miss', 'situation': 'assisted', 'playerCoordinates': {'x': 0.7885750321085476, 'y': 20.704843570335925, 'z': 0}, 'bodyPart': 'right-foot', 'goalMouthLocation': 'close-left', 'goalMouthCoordinates': {'x': 0, 'y': 20.26772880765766, 'z': 26.4}, 'xg': 0.124048627913, 'xgot': 0, 'id': 3706976, 'time': 95, 'addedTime': 5, 'timeSeconds': 5671, 'draw': {'start': {'x': 45.287443154254404, 'y': 0.1911944812343378}, 'end': {'x': 15.85546507314605, 'y': 0}, 'goal': {'x': 86.38495308244313, 'y': 17.034199991098703}}, 'reversedPeriodTime': 1, 'reversedPeriodTimeSeconds': 629, 'incidentType': 'shot'}, {'player': {'name': 'Welington', 'firstName': '', 'lastName': '', 'slug': 'welington', 'shortName': 'Welington', 'position': 'D', 'jerseyNumber': '6', 'userCount': 1127, 'id': 1009920, 'fieldTranslations': {'nameTransla

In [25]:
import pandas as pd

# 'shotmap' listesini DataFrame'e dönüştürme
df = pd.DataFrame(shots['shotmap'])

# 'player' ve 'playerCoordinates' gibi iç içe geçmiş sütunları ayırma
player_df = pd.json_normalize(df['player'])
player_coordinates_df = pd.json_normalize(df['playerCoordinates'])
goal_mouth_coordinates_df = pd.json_normalize(df['goalMouthCoordinates'])
draw_df = pd.json_normalize(df['draw'])

# Bu DataFrame'leri ana DataFrame'e birleştirme
df = pd.concat([df, player_df, player_coordinates_df, goal_mouth_coordinates_df, draw_df], axis=1)

# Gerekli sütunları düşürme
df = df.drop(['player', 'playerCoordinates', 'goalMouthCoordinates', 'draw'], axis=1)

# Yeni sütun isimlerini düzenleme
df.columns = [col.replace('.', '_') for col in df.columns]

print(df.head())

   isHome shotType situation    bodyPart goalMouthLocation        xg  \
0    True     miss  assisted  right-foot        close-left  0.124049   
1    True    block   regular  right-foot         low-right  0.034926   
2    True    block  assisted        head          low-left  0.221591   
3   False     save  assisted   left-foot        low-centre  0.008037   
4    True    block  assisted  right-foot        low-centre  0.026538   

       xgot       id  time  addedTime  ...          y     z    start_x  \
0  0.000000  3706976    95        5.0  ...  20.267729  26.4  45.287443   
1  0.000000  3706975    96        5.0  ...  47.000000  19.0  18.064406   
2  0.000000  3706968    97        5.0  ...  95.028507  19.0   1.006400   
3  0.021231  3706965    89        3.0  ...  35.474019  14.6  10.649829   
4  0.000000  3706961    86        3.0  ...  49.000000  19.0  46.240369   

     start_y      end_x end_y     goal_x    goal_y    block_x    block_y  
0   0.191194  15.855465     0  86.384953   17.0

In [26]:
df.columns

Index(['isHome', 'shotType', 'situation', 'bodyPart', 'goalMouthLocation',
       'xg', 'xgot', 'id', 'time', 'addedTime', 'timeSeconds',
       'reversedPeriodTime', 'reversedPeriodTimeSeconds', 'incidentType',
       'blockCoordinates', 'goalType', 'name', 'slug', 'shortName', 'position',
       'jerseyNumber', 'userCount', 'id', 'firstName', 'lastName',
       'fieldTranslations_nameTranslation_ar',
       'fieldTranslations_shortNameTranslation_ar', 'x', 'y', 'z', 'x', 'y',
       'z', 'start_x', 'start_y', 'end_x', 'end_y', 'goal_x', 'goal_y',
       'block_x', 'block_y'],
      dtype='object')

In [27]:
df[["name", "xg", "time","id", "shotType","userCount", "shortName"]].sort_values(by="time", ascending=False)

,name,xg,time,id,id,shotType,userCount,shortName
2,Luiz Gustavo,0.221591,97,3706968,29735,block,1801,L. Gustavo
7,Luciano,0.035054,96,3706948,282557,block,4838,Luciano
1,Welington,0.034926,96,3706975,1009920,block,1127,Welington
0,André Silva,0.124049,95,3706976,925072,miss,1263,A. Silva
5,Keno,0.053497,93,3706957,787607,goal,1395,Keno
6,William,0.015255,91,3706946,1604944,miss,615,William
3,Sabino,0.008037,89,3706965,975118,save,550,Sabino
4,William,0.026538,86,3706961,1604944,block,615,William
8,Luciano,0.051081,79,3706945,282557,block,4838,Luciano
9,Keno,0.012051,74,3706926,787607,save,1395,Keno
